In [56]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [57]:
compare_folder = "/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2021-02-05_COVID"
pairwise_filename = [x for x in os.listdir(compare_folder) if '.snp.pairwise.tsv' in x][0]
pairwise_file = os.path.join(compare_folder, pairwise_filename)
pwdf = pd.read_csv(pairwise_file, sep="\t" , names=['sample_1', 'sample_2', 'dist'])

In [58]:
pwdf.head()

,sample_1,sample_2,dist
0,20461953,20461953,0
1,20461953,20490128,12
2,20461953,21038641,10
3,20461953,21007127,37
4,20461953,20069944,14


In [59]:
pwdf['sample_1'] = pwdf['sample_1'].astype(str)

In [60]:
pwdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3076516 entries, 0 to 3076515
Data columns (total 3 columns):
sample_1    object
sample_2    object
dist        int64
dtypes: int64(1), object(2)
memory usage: 70.4+ MB


In [61]:
samples = list(set(pwdf.sample_1.tolist()))

In [62]:
len(samples)

1754

In [63]:
samples_type = [type(x) for x in samples]
dict((x,samples_type.count(x)) for x in set(samples_type))

{str: 1754}

In [203]:
wgs = pd.read_csv('/media/NASII/Datos/ANALYSIS/WGS_sequenced.csv')
wgs = wgs.rename(columns={'Muestra': 'id', 'Fecha': 'full_date', 'SEXO' : 'SEX__autocolor', 'EDAD': 'AGE__autocolor', 'Día':'day', 'Mes' : 'month', 'Año':'year'})

In [204]:
wgs.head(2)

,Apellidos,Nombre,NHC,CIPA,AGE__autocolor,SEX__autocolor,CENTRO SANITARIO,id,Sufijo,COV>30X,RUN,DEPARTMENT__autocolor,patient_microreact,full_date,day,month,year,Extracción Muestra,GenN2_Ct,CT1_PCR1,Antígeno COVID
0,¡,FRANCISCO,2198204,NaN,71.122375,M,HOSPITAL GENERAL UNIVERSITARIO GREGORIO MARAÑÓN,20423238,NaN,3.63,RUN_10,NaN,NaN,28/10/2020,28.0,10.0,2020.0,EX. NASAL - FARÍNGEO,17,NaN,NaN
1,AAROUD,AHMIDOU,11031999,NaN,77.000000,M,HOSPITAL GENERAL UNIVERSITARIO GREGORIO MARAÑÓN,20097501,NaN,39.43,RUN_12,NaN,NaN,08/04/2020,8.0,4.0,2020.0,NaN,NaN,NaN,NaN


In [205]:
wgs.columns

Index(['Apellidos', 'Nombre', 'NHC', 'CIPA', 'AGE__autocolor',
       'SEX__autocolor', 'CENTRO SANITARIO', 'id', 'Sufijo', 'COV>30X', 'RUN',
       'DEPARTMENT__autocolor', 'patient_microreact', 'full_date', 'day',
       'month', 'year', 'Extracción Muestra', 'GenN2_Ct', 'CT1_PCR1',
       'Antígeno COVID'],
      dtype='object')

In [206]:
wgs = wgs[['id', 'full_date', 'SEX__autocolor', 'AGE__autocolor', 'day', 'month', 'year']]

In [207]:
wgs.head()

,id,full_date,SEX__autocolor,AGE__autocolor,day,month,year
0,20423238,28/10/2020,M,71.122375,28.0,10.0,2020.0
1,20097501,08/04/2020,M,77.000000,8.0,4.0,2020.0
2,20395396,9/10/2020,M,80.210930,9.0,10.0,2020.0
3,20091284,2/4/2020,M,NaN,2.0,4.0,2020.0
4,20432945,4/11/2020,F,28.247288,4.0,11.0,2020.0


In [208]:
wgs.shape

(2007, 7)

In [209]:
wgs['id'] = wgs['id'].astype(str)

In [210]:
samples_wgs = wgs.id.tolist()
#samples_wgs = [str(x) for x in samples_wgs]

In [211]:
WGS = set(samples_wgs)
PW = set(samples)

In [212]:
len(PW)

1754

In [213]:
missing_wgs = list(PW - WGS)
len(missing_wgs)

118

In [214]:
missing_wgs_type = [type(x) for x in missing_wgs]
dict((x,missing_wgs_type.count(x)) for x in set(missing_wgs_type))

{str: 118}

In [215]:
missing_pw = list(WGS - PW)
len(missing_pw)

239

In [216]:
all_missing = missing_pw + missing_wgs
len(all_missing)

357

In [217]:
wgs.shape

(2007, 7)

In [218]:
df = wgs[~wgs.id.isin(missing_pw)]

In [219]:
df.shape

(1733, 7)

In [220]:
df2 = pd.DataFrame(missing_wgs, columns=['id'])

In [221]:
df2['id'] = df2['id'].astype(str)

In [222]:
df2.head()

,id
0,20431685BIS
1,20461087BIS
2,20461238BIS
3,20164346-8
4,20466272BIS


In [223]:
df.head()

,id,full_date,SEX__autocolor,AGE__autocolor,day,month,year
3,20091284,2/4/2020,M,NaN,2.0,4.0,2020.0
4,20432945,4/11/2020,F,28.247288,4.0,11.0,2020.0
5,21035911,23/1/2021,F,49.521111,23.0,1.0,2021.0
6,21020388,15/01/2021,M,76.000000,15.0,1.0,2021.0
8,20089098,1/4/2020,F,NaN,1.0,4.0,2020.0


In [224]:
final_df = df.append(df2).reset_index(drop=True)
final_df = final_df[ ['id', 'full_date', 'SEX__autocolor', 'AGE__autocolor', 'day', 'month', 'year'] ]


In [225]:
final_df.head()

,id,full_date,SEX__autocolor,AGE__autocolor,day,month,year
0,20091284,2/4/2020,M,NaN,2.0,4.0,2020.0
1,20432945,4/11/2020,F,28.247288,4.0,11.0,2020.0
2,21035911,23/1/2021,F,49.521111,23.0,1.0,2021.0
3,21020388,15/01/2021,M,76.000000,15.0,1.0,2021.0
4,20089098,1/4/2020,F,NaN,1.0,4.0,2020.0


In [226]:
metadata = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/wgs_covid_metadata.csv', encoding='latin1')
metadata['id'] = metadata['id'].astype(str)

In [227]:
metadata.tail()

,id,DEPARTMENT__autocolor
1214,21046824,PSIQUIATRÃA
1215,21050677,PSIQUIATRÃA
1216,20296976,REINFECCION 52
1217,21037795,UK princesa
1218,20082339,ReinfecciÃ³n PPCA


In [228]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1219 entries, 0 to 1218
Data columns (total 2 columns):
id                       1219 non-null object
DEPARTMENT__autocolor    1219 non-null object
dtypes: object(2)
memory usage: 19.2+ KB


In [229]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1851 entries, 0 to 1850
Data columns (total 7 columns):
id                1851 non-null object
full_date         1730 non-null object
SEX__autocolor    1590 non-null object
AGE__autocolor    1655 non-null float64
day               1730 non-null float64
month             1730 non-null float64
year              1730 non-null float64
dtypes: float64(4), object(3)
memory usage: 101.4+ KB


In [230]:
final_df = final_df.fillna(0)
final_df.day = final_df.day.astype('Int64')
final_df.month = final_df.month.astype('Int64')
final_df.year = final_df.year.astype('Int64')
final_df.replace(0, np.nan, inplace=True)

In [231]:
final_df = final_df.merge(metadata, how='outer')

In [232]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1966 entries, 0 to 1965
Data columns (total 8 columns):
id                       1966 non-null object
full_date                1740 non-null object
SEX__autocolor           1600 non-null object
AGE__autocolor           1665 non-null float64
day                      1740 non-null Int64
month                    1740 non-null Int64
year                     1740 non-null Int64
DEPARTMENT__autocolor    1316 non-null object
dtypes: Int64(3), float64(1), object(4)
memory usage: 144.0+ KB


In [233]:
final_df.tail()

,id,full_date,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor
1961,21050541,NaN,NaN,NaN,NaN,NaN,NaN,BROTE 2201
1962,21046256,NaN,NaN,NaN,NaN,NaN,NaN,PSIQUIATRÃA
1963,20296976,NaN,NaN,NaN,NaN,NaN,NaN,REINFECCION 52
1964,21037795,NaN,NaN,NaN,NaN,NaN,NaN,UK princesa
1965,20082339,NaN,NaN,NaN,NaN,NaN,NaN,ReinfecciÃ³n PPCA


In [234]:
def extract_lineage(sample):
    pango_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/pangolin', str(sample) + ".lineage.csv")
    if os.path.exists(pango_file):
        df = pd.read_csv(pango_file)
        return df.lineage[0]
    else:
        return None

In [235]:
final_df['Lineage__autocolor'] = final_df.apply(lambda x: extract_lineage(x['id']), axis=1)

In [236]:
def extract_EU(sample):
    eu_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user', str(sample) + ".tsv")
    if os.path.exists(eu_file):
        df = pd.read_csv(eu_file, sep="\t")
        if 'Nextrain_all' in df.columns:
            eulist = df.Nextrain_all.dropna().tolist()
            return (',').join([(x + "-" + str(eulist.count(x))) for x in set(eulist)])
        else:
            return None

In [237]:
final_df['Nextrain__autocolor'] = final_df.apply(lambda x: extract_EU(x['id']), axis=1)

In [238]:
def extract_resistance(sample):
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'resistance_covid' in df.columns:
            relist = df.resistance_covid.dropna().tolist()
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

In [239]:
final_df['Resistance__autocolor'] = final_df.apply(lambda x: extract_resistance(x['id']), axis=1)

In [240]:
def extract_variant(sample):
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'Variants' in df.columns:
            df = df.drop_duplicates(subset=['Variants'], keep="first")
            relist = df.Variants.dropna().tolist()
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

In [241]:
final_df['VariantsAll__autocolor'] = final_df.apply(lambda x: extract_variant(x['id']), axis=1)

In [242]:
def extract_variant_count(sample):
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'Variants' in df.columns:
            df = df.drop_duplicates(subset=['Variants'], keep="first")
            relist = df.Variants.dropna().tolist()
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

In [243]:
final_df['VariantsCount__autocolor'] = final_df.apply(lambda x: extract_variant_count(x['id']), axis=1)

In [244]:
def extract_variant_UK(sample):
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'UKVariant' in df.columns: #Change header
            df = df.drop_duplicates(subset=['UKVariant'], keep="first") #Change header
            relist = df.UKVariant.dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

def extract_variant_D(sample):
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'DVariant' in df.columns: #Change header
            df = df.drop_duplicates(subset=['DVariant'], keep="first") #Change header
            relist = df.DVariant.dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

def extract_variant_69del(sample):
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if '69del' in df.columns: #Change header
            df = df.drop_duplicates(subset=['69del'], keep="first") #Change header
            relist = df['69del'].dropna().tolist() #Change header
            relist = [x.split(":")[1] for x in relist]
            return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
        else:
            return None

In [245]:
final_df['UKVariant__autocolor'] = final_df.apply(lambda x: extract_variant_UK(x['id']), axis=1)
final_df['DVariant__autocolor'] = final_df.apply(lambda x: extract_variant_D(x['id']), axis=1)
final_df['69del__autocolor'] = final_df.apply(lambda x: extract_variant_69del(x['id']), axis=1)

In [246]:
def extract_faulty(sample):
    sample_report = os.path.join(compare_folder, "report_samples.tsv")
    df = pd.read_csv(sample_report, sep="\t")
    df['sample'] = df['sample'].astype(str)
    if str(sample) in df['sample'].values:
        return round(df[df['sample'] == str(sample)]['faulty_frac'].values[0], 3)
    else:
        return None

In [247]:
final_df['FaultyFraction__autocolor'] = final_df.apply(lambda x: extract_faulty(x['id']), axis=1)

In [248]:
#query = ['20097501','20097906','20099990','20101024','20107580','20143658','20214237','20247923','20398264','20425128','20425595','20428122','20431031','20435733','20450595','20450832','20455085','20455763','20455870','20458052','20458113','20458114','20458118','20458269','20458270','20458365','20458528','20458547','20458560','20458569','20458576','20458635','20458638','20458668','20458686','20458702','20461287','20461287-BIS','20461290','20461293','20461349','20461351','20461375','20461389','20461389-BIS','20461392','20461416','20461563','20461601','20461621','20461637','20461637-BIS','20461689','20461747','20461783','20461785','20461790','20461801','20461861','20461869','20461870','20461871','20461886','20461904','20461905','20461907','20461908','20461926','20461928','20461937','20461953','20461968','20464095','20464109','20464109-BIS','20464197','20464303','20464456','20464490','20464493','20464516','20464517','20464519','20464521','20464532','20464534','20464567','20464570','20464580','20464595','20464599','20464600','20464612','20464613','20464614','20464636']
#query_dario = [str(x) for x in query_dario]


In [249]:
#final_df['run12Maldito__autocolor'] = final_df.apply(lambda x: x['id'] in query, axis=1)


In [250]:
before_june = final_df[final_df.month < 7]['id'].tolist()
before_june = [str(x) for x in before_june]
final_df['before_june_2020__autocolor'] = final_df.apply(lambda x: x['id'] in before_june, axis=1)

In [251]:
final_df = final_df.drop_duplicates(subset=['id'], keep="first")
#REORDER COLUMNS
final_df = final_df[['DEPARTMENT__autocolor'] + [ col for col in final_df.columns if col != 'DEPARTMENT__autocolor']]

out_file = os.path.join(compare_folder, 'wgs_microreact.csv')
final_df.to_csv(out_file, index=False)

In [252]:
final_df.head()

,DEPARTMENT__autocolor,id,full_date,SEX__autocolor,AGE__autocolor,day,month,year,Lineage__autocolor,Nextrain__autocolor,Resistance__autocolor,VariantsAll__autocolor,VariantsCount__autocolor,UKVariant__autocolor,DVariant__autocolor,69del__autocolor,FaultyFraction__autocolor,before_june_2020__autocolor
0,NaN,20091284,2/4/2020,M,NaN,2,4,2020,None,20A-2,,,,None,None,None,0.001,True
1,TRABAJADORES_OCT_NOV,20432945,4/11/2020,F,28.247288,4,11,2020,B.1.177,"20A-2,20AEU1-3",,,,None,None,None,0.001,False
2,Trabajadores,21035911,23/1/2021,F,49.521111,23,1,2021,B.1.177,"20A-2,20AEU1-3",,,,None,None,None,0.001,True
3,SOSPECHA UK,21020388,15/01/2021,M,76.000000,15,1,2021,None,"20B-3,20A-2",,"Ala1708Asp:UKVariant-1,Ile2230Thr:UKVariant-1,Thr716Ile:UKVariant,DVariant-1,Ala570Asp:UKVariant-1,Arg52Ile:UKVariant-1,Ser982Ala:UKVariant-1,Ser235Phe:UKVariant-1,Asn501Tyr:UKVariant,ZAVariant,P1Variant-1,Gly3676del:UKVariant,P1Variant-1,Gln27*:UKVariant-1,Thr1001Ile:UKVariant-1,His69del:UKVariant-1,Tyr73Cys:UKVariant-1,Pro681His:UKVariant,DVariant-1,Asp1118His:UKVariant-1,Tyr144del:UKVariant-1","UKVariant,ZAVariant,P1Variant-1,UKVariant-12,UKVariant,P1Variant-1,UKVariant,DVariant-2",None,None,None,0.005,True
4,NaN,20089098,1/4/2020,F,NaN,1,4,2020,None,19B-3,,,,None,None,None,0.000,True
